# Flow SDK Configuration & Authentication

This notebook covers everything you need to know about configuring Flow SDK, managing authentication, and customizing your environment.

## What You'll Learn

1. **Flow Configuration** - Understanding `flow init` and config files
2. **Authentication** - API keys, credentials, and security
3. **Project Configuration** - Project-specific settings
4. **SSH Key Management** - Accessing instances directly
5. **Environment Variables** - Advanced configuration options
6. **Provider Selection** - Choosing compute backends

In [ ]:
# Import required modules
import json
import os
from pathlib import Path

from flow import Flow, TaskConfig
from flow.config import FlowConfig

## 1. Understanding Flow Configuration

Flow SDK uses a hierarchical configuration system:
1. Default values
2. User config file (`~/.flow/config.yaml`)
3. Project config file (`.flow/config.yaml`)
4. Environment variables
5. Runtime parameters

In [ ]:
# Check current configuration
config = FlowConfig()

print("Current Flow Configuration:")
print(f"  API URL: {config.api_url}")
print(f"  Provider: {config.provider}")
print(f"  Project: {config.project}")
print(f"  Region: {config.region}")
print(f"  Config Path: {config.config_path}")

In [ ]:
# Check for configuration files
user_config = Path.home() / ".flow" / "config.yaml"
project_config = Path.cwd() / ".flow" / "config.yaml"

print("Configuration Files:")
print(f"  User config: {user_config} ({'exists' if user_config.exists() else 'not found'})")
print(
    f"  Project config: {project_config} ({'exists' if project_config.exists() else 'not found'})"
)

# Display user config if it exists
if user_config.exists():
    print("\nUser Configuration:")
    with open(user_config) as f:
        print(f.read())

## 2. Authentication Methods

Flow SDK supports multiple authentication methods depending on the provider.

In [ ]:
# Check authentication status
from flow.auth import Authenticator

auth = Authenticator()

# Check if authenticated
try:
    # Attempt to get credentials
    creds = auth.get_credentials()
    print("Authentication Status: ✓ Authenticated")
    print(f"Provider: {config.provider}")

    # Show credential type (without exposing secrets)
    if "api_key" in creds:
        print("Credential Type: API Key")
        print(f"API Key: {'*' * 32}{creds['api_key'][-4:]}")
    elif "token" in creds:
        print("Credential Type: Bearer Token")
        print(f"Token: {'*' * 32}{creds['token'][-4:]}")
except Exception as e:
    print("Authentication Status: ✗ Not authenticated")
    print(f"Error: {e}")
    print("\nRun 'flow init' to configure authentication")

In [ ]:
# Environment variables for authentication
auth_env_vars = [
    "FLOW_API_KEY",
    "FLOW_API_URL",
    "FLOW_PROVIDER",
    "FLOW_PROJECT",
    "FLOW_REGION",
    "Mithril_API_KEY",  # Provider-specific
]

print("Authentication Environment Variables:")
for var in auth_env_vars:
    value = os.environ.get(var)
    if value:
        if "KEY" in var or "TOKEN" in var:
            # Mask sensitive values
            display_value = f"{'*' * 32}{value[-4:]}"
        else:
            display_value = value
        print(f"  {var}: {display_value}")
    else:
        print(f"  {var}: (not set)")

## 3. Project-Specific Configuration

You can create project-specific configurations that override global settings.

In [ ]:
# Create a sample project configuration
project_config_example = {
    "provider": "mithril",
    "project": "ml-research",
    "region": "us-west-2",
    "defaults": {
        "instance_type": "gpu.nvidia.a10g",
        "max_price_per_hour": 15.0,
        "volumes": [{"name": "project-data", "size_gb": 100, "mount_path": "/data"}],
    },
    "ssh_keys": ["ssh-key-123", "ssh-key-456"],
    "environment": {"CUDA_VISIBLE_DEVICES": "0,1", "PYTHONPATH": "/workspace/src"},
}

print("Example Project Configuration:")
print(json.dumps(project_config_example, indent=2))

In [ ]:
# Demonstrate how project config affects task creation
# This would use the project defaults if .flow/config.yaml exists

# Minimal config - uses project defaults
minimal_config = TaskConfig(name="project-task", command="python train.py")

print("Minimal TaskConfig:")
print(f"  Name: {minimal_config.name}")
print(f"  Instance Type: {minimal_config.instance_type or 'Will use project default'}")
print(f"  Max Price: {minimal_config.max_price_per_hour or 'Will use project default'}")

# Override project defaults
override_config = TaskConfig(
    name="override-task",
    instance_type="gpu.nvidia.h100",  # Override default A10G
    max_price_per_hour=50.0,  # Override default $15
    command="python train_large_model.py",
)

print("\nOverride TaskConfig:")
print(f"  Name: {override_config.name}")
print(f"  Instance Type: {override_config.instance_type}")
print(f"  Max Price: ${override_config.max_price_per_hour}")

## 4. SSH Key Management

Flow SDK allows you to configure SSH keys for direct instance access.

In [ ]:
# Check for SSH keys
ssh_dir = Path.home() / ".ssh"
ssh_keys = []

if ssh_dir.exists():
    # Look for public keys
    for key_file in ssh_dir.glob("*.pub"):
        ssh_keys.append(key_file.name)

print("Available SSH Public Keys:")
if ssh_keys:
    for key in ssh_keys:
        print(f"  - {key}")
else:
    print("  No SSH public keys found")
    print("  Generate one with: ssh-keygen -t ed25519 -C 'your-email@example.com'")

In [ ]:
# Example: Task with SSH access enabled
ssh_config = TaskConfig(
    name="ssh-enabled-task",
    instance_type="gpu.nvidia.t4",
    ssh_keys=["ssh-key-id-123"],  # Your SSH key IDs from the provider
    ports=[22],  # Enable SSH port
    command="""
        echo "Instance ready for SSH access"
        echo "Connect using: ssh ubuntu@$(curl -s ifconfig.me)"
        
        # Keep instance alive
        while true; do
            echo "Instance running... ($(date))"
            sleep 60
        done
    """,
)

print("SSH-Enabled Task Configuration:")
print(f"  SSH Keys: {ssh_config.ssh_keys}")
print(f"  Open Ports: {ssh_config.ports}")

## 5. Environment Variables and Runtime Configuration

Flow SDK supports various environment variables for configuration and runtime behavior.

In [ ]:
# All Flow environment variables
flow_env_vars = {
    # Authentication
    "FLOW_API_KEY": "API key for authentication",
    "FLOW_API_URL": "API endpoint URL",
    # Provider Configuration
    "FLOW_PROVIDER": "Provider to use (mithril, aws, gcp, azure)",
    "FLOW_PROJECT": "Default project name",
    "FLOW_REGION": "Default region",
    # Runtime Behavior
    "FLOW_LOG_LEVEL": "Logging level (DEBUG, INFO, WARNING, ERROR)",
    "FLOW_TIMEOUT": "Default timeout in seconds",
    "FLOW_RETRY_COUNT": "Number of retries for failed operations",
    # Task Defaults
    "FLOW_DEFAULT_INSTANCE_TYPE": "Default instance type",
    "FLOW_DEFAULT_MAX_PRICE": "Default max price per hour",
    # Development
    "FLOW_DEBUG": "Enable debug mode (true/false)",
    "FLOW_DRY_RUN": "Preview operations without executing",
}

print("Flow Environment Variables Reference:\n")
for var, desc in flow_env_vars.items():
    current = os.environ.get(var, "(not set)")
    print(f"{var}:")
    print(f"  Description: {desc}")
    print(f"  Current: {current}")
    print()

In [ ]:
# Example: Using environment variables in tasks
env_task_config = TaskConfig(
    name="env-demo",
    instance_type="cpu.small",
    environment={"MY_API_KEY": "secret-key-123", "MODEL_PATH": "/models/latest", "DEBUG": "true"},
    command="""
        echo "=== Task Environment Variables ==="
        echo "MY_API_KEY: $MY_API_KEY"
        echo "MODEL_PATH: $MODEL_PATH"
        echo "DEBUG: $DEBUG"
        echo ""
        echo "=== Flow Environment Variables ==="
        env | grep ^FLOW_ | sort
    """,
)

print("Task with custom environment variables:")
print(f"Environment: {json.dumps(env_task_config.environment, indent=2)}")

## 6. Provider Selection and Configuration

Flow SDK supports multiple compute providers. Each has specific configuration requirements.

In [ ]:
# Provider configurations
providers = {
    "mithril": {
        "name": "Mithril (Flow Compute Platform)",
        "auth": "API Key",
        "env_vars": ["Mithril_API_KEY", "Mithril_API_URL"],
        "features": ["Spot instances", "Auto-bidding", "Global regions"],
    },
    "aws": {
        "name": "Amazon Web Services",
        "auth": "AWS Credentials",
        "env_vars": ["AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY", "AWS_REGION"],
        "features": ["EC2", "EKS", "Spot instances"],
    },
    "gcp": {
        "name": "Google Cloud Platform",
        "auth": "Service Account",
        "env_vars": ["GOOGLE_APPLICATION_CREDENTIALS", "GCP_PROJECT"],
        "features": ["Compute Engine", "GKE", "Preemptible VMs"],
    },
    "azure": {
        "name": "Microsoft Azure",
        "auth": "Azure CLI",
        "env_vars": ["AZURE_SUBSCRIPTION_ID", "AZURE_TENANT_ID"],
        "features": ["Virtual Machines", "AKS", "Spot instances"],
    },
}

print("Supported Providers:\n")
for key, info in providers.items():
    print(f"{key}: {info['name']}")
    print(f"  Authentication: {info['auth']}")
    print(f"  Environment: {', '.join(info['env_vars'])}")
    print(f"  Features: {', '.join(info['features'])}")
    print()

In [ ]:
# Example: Switching providers at runtime
# Note: This requires appropriate credentials for each provider

# Default provider (from config)
with Flow() as flow:
    print(f"Default provider: {flow.provider}")

# Override provider
try:
    with Flow(provider="mithril") as flow:
        print(f"Override provider: {flow.provider}")
except Exception as e:
    print(f"Error: {e}")
    print("Note: Provider override requires valid credentials")

## 7. Advanced Configuration Patterns

Let's explore some advanced configuration patterns for production use.

In [ ]:
# Configuration inheritance example
base_config = {
    "provider": "mithril",
    "defaults": {
        "max_price_per_hour": 10.0,
        "auto_terminate": True,
        "environment": {"PYTHONUNBUFFERED": "1", "TZ": "UTC"},
    },
}

# Team-specific config (extends base)
team_config = {
    **base_config,
    "project": "ml-team-alpha",
    "defaults": {
        **base_config["defaults"],
        "instance_type": "gpu.nvidia.a10g",
        "volumes": [
            {"name": "team-data", "size_gb": 500, "mount_path": "/data"},
            {"name": "team-models", "size_gb": 200, "mount_path": "/models"},
        ],
    },
}

# Production config (extends team)
prod_config = {
    **team_config,
    "defaults": {
        **team_config["defaults"],
        "max_price_per_hour": 50.0,  # Higher limit for production
        "instance_type": "gpu.nvidia.h100",  # Better hardware
        "environment": {
            **team_config["defaults"]["environment"],
            "ENV": "production",
            "LOG_LEVEL": "INFO",
        },
    },
}

print("Configuration Hierarchy:\n")
print("Base Config:")
print(json.dumps(base_config, indent=2))
print("\nTeam Config (extends base):")
print(json.dumps(team_config, indent=2))
print("\nProduction Config (extends team):")
print(json.dumps(prod_config, indent=2))

In [ ]:
# Multi-environment configuration strategy
import os


def get_environment_config():
    """Get configuration based on environment."""
    env = os.environ.get("FLOW_ENV", "development")

    configs = {
        "development": {"instance_type": "gpu.nvidia.t4", "max_price_per_hour": 5.0, "debug": True},
        "staging": {"instance_type": "gpu.nvidia.a10g", "max_price_per_hour": 20.0, "debug": False},
        "production": {
            "instance_type": "gpu.nvidia.a100",
            "max_price_per_hour": 50.0,
            "debug": False,
            "monitoring": True,
        },
    }

    return configs.get(env, configs["development"])


# Example usage
for env in ["development", "staging", "production"]:
    os.environ["FLOW_ENV"] = env
    config = get_environment_config()
    print(f"{env.capitalize()} Configuration:")
    print(json.dumps(config, indent=2))
    print()

## 8. Security Best Practices

Follow these security best practices when configuring Flow SDK.

In [ ]:
# Security checklist
security_checks = [
    {
        "check": "API keys in environment variables",
        "good": "export FLOW_API_KEY='your-key'",
        "bad": "api_key = 'your-key'  # In code",
    },
    {
        "check": "Credentials in version control",
        "good": "Add .flow/ to .gitignore",
        "bad": "Commit config.yaml with API keys",
    },
    {
        "check": "SSH key management",
        "good": "Use provider's SSH key management",
        "bad": "Embed private keys in scripts",
    },
    {
        "check": "Secrets in tasks",
        "good": "Pass via environment variables",
        "bad": "Hardcode in command strings",
    },
    {
        "check": "Network security",
        "good": "Open only required ports",
        "bad": "ports=[22, 80, 443, 3306, 5432]  # Too many",
    },
]

print("Security Best Practices:\n")
for item in security_checks:
    print(f"✓ {item['check']}")
    print(f"  Good: {item['good']}")
    print(f"  Bad:  {item['bad']}")
    print()

In [ ]:
# Example: Secure task configuration
secure_config = TaskConfig(
    name="secure-task",
    instance_type="gpu.nvidia.a10g",
    # Pass secrets via environment
    environment={
        "DATABASE_URL": os.environ.get("DATABASE_URL", ""),
        "API_TOKEN": os.environ.get("API_TOKEN", ""),
        "S3_BUCKET": os.environ.get("S3_BUCKET", ""),
    },
    # Minimal port exposure
    ports=[443],  # Only HTTPS
    # Use startup script for sensitive operations
    command="""
        # Validate environment
        if [ -z "$DATABASE_URL" ]; then
            echo "ERROR: DATABASE_URL not set"
            exit 1
        fi
        
        # Run application
        python app.py --secure
    """,
)

print("Secure Task Configuration:")
print(f"  Environment variables: {len(secure_config.environment)}")
print(f"  Open ports: {secure_config.ports}")
print("  Validation: Built into startup script")

## 9. Troubleshooting Configuration Issues

Common configuration issues and how to resolve them.

In [ ]:
# Configuration debugging helper
def debug_flow_config():
    """Debug Flow SDK configuration issues."""
    issues = []

    # Check authentication
    if not os.environ.get("FLOW_API_KEY"):
        issues.append("FLOW_API_KEY not set")

    # Check config files
    user_config = Path.home() / ".flow" / "config.yaml"
    if not user_config.exists():
        issues.append("User config not found - run 'flow init'")

    # Check provider
    provider = os.environ.get("FLOW_PROVIDER", "")
    if provider and provider not in ["mithril", "aws", "gcp", "azure"]:
        issues.append(f"Unknown provider: {provider}")

    # Check permissions
    flow_dir = Path.home() / ".flow"
    if flow_dir.exists() and not os.access(flow_dir, os.W_OK):
        issues.append(f"No write permission for {flow_dir}")

    return issues


# Run diagnostic
issues = debug_flow_config()

print("Configuration Diagnostic:\n")
if issues:
    print("Issues found:")
    for i, issue in enumerate(issues, 1):
        print(f"  {i}. {issue}")
else:
    print("✓ No configuration issues detected")

# Show current effective configuration
print("\nEffective Configuration:")
try:
    config = FlowConfig()
    print(f"  Provider: {config.provider}")
    print(f"  API URL: {config.api_url}")
    print(f"  Project: {config.project}")
    print(f"  Region: {config.region}")
except Exception as e:
    print(f"  Error loading config: {e}")

## Summary and Next Steps

You've learned how to:
- Configure Flow SDK at user and project levels
- Manage authentication and credentials securely
- Use environment variables for runtime configuration
- Set up SSH access to instances
- Switch between providers
- Follow security best practices

### Quick Reference

```bash
# Initialize Flow
flow init

# Set API key
export FLOW_API_KEY="your-key-here"

# Use specific provider
export FLOW_PROVIDER="mithril"

# Enable debug logging
export FLOW_LOG_LEVEL="DEBUG"
```

### Next Notebooks

- **Notebook 3**: Multiple Frontends - YAML, SLURM, and more
- **Notebook 4**: Advanced Features - Catalogs, multi-node, and monitoring
- **Notebook 5**: Real-World Examples - Complete ML workflows